# LensPR Benchmark: With vs Without

Measures code reliability and context usage when using Claude with LensPR tools vs standard tools.

## Metrics
- **Input tokens** — context burned
- **Output tokens** — generated
- **Iterations** — conversation turns
- **Tool calls** — which tools and how many
- **Success** — task completed correctly?

## Setup

In [1]:
import sys
sys.path.insert(0, '..')

import json
import os
from pathlib import Path
from dataclasses import dataclass, field, asdict
from datetime import datetime
from typing import Any

# Load API key from .env
from dotenv import load_dotenv
load_dotenv()

import anthropic
import lenspr

# Initialize
PROJECT_ROOT = Path('..').resolve()
RESULTS_DIR = Path('results')
RESULTS_DIR.mkdir(exist_ok=True)

client = anthropic.Anthropic()  # Uses ANTHROPIC_API_KEY from env
ctx = lenspr.init(str(PROJECT_ROOT))

print(f"Project: {PROJECT_ROOT}")
print(f"LensPR initialized: {ctx is not None}")
print(f"API key loaded: {'ANTHROPIC_API_KEY' in os.environ}")

Project: /Users/kyryloprymak/code/lenspr
LensPR initialized: True
API key loaded: True


## Metrics Tracking

In [2]:
@dataclass
class BenchmarkResult:
    """Stores results from a single benchmark run."""
    task_id: str
    mode: str  # 'with_lenspr' or 'without_lenspr'
    
    # Token metrics
    total_input_tokens: int = 0
    total_output_tokens: int = 0
    
    # Iteration metrics
    iterations: int = 0
    
    # Tool usage
    tool_calls: list = field(default_factory=list)
    tool_call_count: int = 0
    
    # Success metrics
    completed: bool = False
    error: str | None = None
    
    # Timing
    started_at: str = ""
    finished_at: str = ""
    duration_seconds: float = 0.0
    
    # Conversation log
    messages: list = field(default_factory=list)
    
    def save(self, path: Path):
        """Save results to JSON."""
        with open(path, 'w') as f:
            json.dump(asdict(self), f, indent=2, default=str)
    
    @classmethod
    def load(cls, path: Path) -> 'BenchmarkResult':
        """Load results from JSON."""
        with open(path) as f:
            data = json.load(f)
        return cls(**data)

## Tool Definitions

In [3]:
# Standard tools (without LensPR)
STANDARD_TOOLS = [
    {
        "name": "read_file",
        "description": "Read a file from the filesystem. Returns the file content.",
        "input_schema": {
            "type": "object",
            "properties": {
                "path": {"type": "string", "description": "Path to the file to read"}
            },
            "required": ["path"]
        }
    },
    {
        "name": "write_file",
        "description": "Write content to a file. Creates or overwrites the file.",
        "input_schema": {
            "type": "object",
            "properties": {
                "path": {"type": "string", "description": "Path to the file"},
                "content": {"type": "string", "description": "Content to write"}
            },
            "required": ["path", "content"]
        }
    },
    {
        "name": "search_files",
        "description": "Search for a pattern in files using grep. Returns matching lines with file paths.",
        "input_schema": {
            "type": "object",
            "properties": {
                "pattern": {"type": "string", "description": "Pattern to search for"},
                "path": {"type": "string", "description": "Directory to search in", "default": "."}
            },
            "required": ["pattern"]
        }
    },
    {
        "name": "list_files",
        "description": "List files in a directory.",
        "input_schema": {
            "type": "object",
            "properties": {
                "path": {"type": "string", "description": "Directory path", "default": "."}
            }
        }
    },
    {
        "name": "run_tests",
        "description": "Run pytest on the project. Returns test output.",
        "input_schema": {
            "type": "object",
            "properties": {
                "path": {"type": "string", "description": "Test path", "default": "tests/"}
            }
        }
    },
    {
        "name": "task_complete",
        "description": "Call this when the task is complete. Provide a summary of what was done.",
        "input_schema": {
            "type": "object",
            "properties": {
                "summary": {"type": "string", "description": "Summary of completed work"}
            },
            "required": ["summary"]
        }
    }
]

# LensPR tools (from lenspr package)
LENSPR_TOOLS = lenspr.get_claude_tools() + STANDARD_TOOLS

print(f"Standard tools: {len(STANDARD_TOOLS)}")
print(f"LensPR tools: {len(LENSPR_TOOLS)} ({len(LENSPR_TOOLS) - len(STANDARD_TOOLS)} lens_* + {len(STANDARD_TOOLS)} standard)")

Standard tools: 6
LensPR tools: 34 (28 lens_* + 6 standard)


## Tool Handlers

In [4]:
import subprocess

def handle_standard_tool(name: str, inputs: dict) -> str:
    """Handle standard tool calls."""
    try:
        if name == "read_file":
            path = PROJECT_ROOT / inputs["path"]
            if not path.exists():
                return f"Error: File not found: {inputs['path']}"
            return path.read_text()
        
        elif name == "write_file":
            path = PROJECT_ROOT / inputs["path"]
            path.parent.mkdir(parents=True, exist_ok=True)
            path.write_text(inputs["content"])
            return f"Successfully wrote {len(inputs['content'])} characters to {inputs['path']}"
        
        elif name == "search_files":
            search_path = PROJECT_ROOT / inputs.get("path", ".")
            result = subprocess.run(
                ["grep", "-rn", inputs["pattern"], str(search_path)],
                capture_output=True, text=True, timeout=30
            )
            output = result.stdout or "No matches found"
            # Limit output size
            if len(output) > 10000:
                output = output[:10000] + f"\n... (truncated, {len(output)} total chars)"
            return output
        
        elif name == "list_files":
            path = PROJECT_ROOT / inputs.get("path", ".")
            if not path.exists():
                return f"Error: Directory not found: {inputs.get('path', '.')}"
            files = []
            for p in sorted(path.rglob("*.py"))[:100]:  # Limit to 100 files
                rel = p.relative_to(PROJECT_ROOT)
                files.append(str(rel))
            return "\n".join(files) if files else "No Python files found"
        
        elif name == "run_tests":
            test_path = inputs.get("path", "tests/")
            result = subprocess.run(
                ["python", "-m", "pytest", test_path, "-v", "--tb=short"],
                capture_output=True, text=True, timeout=120, cwd=str(PROJECT_ROOT)
            )
            output = result.stdout + result.stderr
            if len(output) > 15000:
                output = output[:15000] + "\n... (truncated)"
            return output
        
        elif name == "task_complete":
            return f"TASK_COMPLETE: {inputs.get('summary', 'No summary')}"
        
        else:
            return f"Unknown tool: {name}"
    
    except Exception as e:
        return f"Error executing {name}: {str(e)}"


def handle_tool_call(name: str, inputs: dict) -> str:
    """Route tool call to appropriate handler."""
    if name.startswith("lens_"):
        result = lenspr.handle_tool(name, inputs)
        return json.dumps(result, indent=2, default=str)
    else:
        return handle_standard_tool(name, inputs)

# Test
print(handle_standard_tool("list_files", {"path": "lenspr"})[:500])

lenspr/__init__.py
lenspr/claude_tools.py
lenspr/cli.py
lenspr/context.py
lenspr/database.py
lenspr/graph.py
lenspr/mcp_server.py
lenspr/models.py
lenspr/parsers/__init__.py
lenspr/parsers/base.py
lenspr/parsers/python_parser.py
lenspr/patcher.py
lenspr/plugins/__init__.py
lenspr/tools/__init__.py
lenspr/tools/analysis.py
lenspr/tools/annotation.py
lenspr/tools/explain.py
lenspr/tools/git.py
lenspr/tools/helpers.py
lenspr/tools/modification.py
lenspr/tools/navigation.py
lenspr/tools/schemas.py
l


## Agentic Loop

In [5]:
def run_agent(
    task: str,
    tools: list,
    system_prompt: str,
    max_iterations: int = 20,
    model: str = "claude-sonnet-4-20250514"
) -> BenchmarkResult:
    """
    Run Claude agent until task_complete is called or max_iterations reached.
    
    Returns BenchmarkResult with all metrics.
    """
    result = BenchmarkResult(
        task_id="",
        mode="with_lenspr" if any(t["name"].startswith("lens_") for t in tools) else "without_lenspr",
        started_at=datetime.now().isoformat()
    )
    
    messages = [{"role": "user", "content": task}]
    
    for iteration in range(max_iterations):
        result.iterations += 1
        print(f"\n--- Iteration {result.iterations} ---")
        
        # Call Claude
        response = client.messages.create(
            model=model,
            max_tokens=4096,
            system=system_prompt,
            tools=tools,
            messages=messages
        )
        
        # Track tokens
        result.total_input_tokens += response.usage.input_tokens
        result.total_output_tokens += response.usage.output_tokens
        
        print(f"Tokens: +{response.usage.input_tokens} in, +{response.usage.output_tokens} out")
        
        # Process response
        assistant_content = response.content
        messages.append({"role": "assistant", "content": assistant_content})
        
        # Check for text output
        for block in assistant_content:
            if hasattr(block, 'text'):
                print(f"Claude: {block.text[:200]}..." if len(block.text) > 200 else f"Claude: {block.text}")
        
        # Handle tool use
        if response.stop_reason == "tool_use":
            tool_results = []
            
            for block in assistant_content:
                if block.type == "tool_use":
                    tool_name = block.name
                    tool_input = block.input
                    
                    result.tool_calls.append({"name": tool_name, "input": tool_input})
                    result.tool_call_count += 1
                    
                    print(f"Tool: {tool_name}({json.dumps(tool_input)[:100]}...)")
                    
                    # Check for task completion
                    if tool_name == "task_complete":
                        result.completed = True
                        result.finished_at = datetime.now().isoformat()
                        result.messages = [{"role": m["role"], "content": str(m["content"])[:500]} for m in messages]
                        print(f"\n✅ Task completed: {tool_input.get('summary', '')}")
                        return result
                    
                    # Execute tool
                    tool_result = handle_tool_call(tool_name, tool_input)
                    tool_results.append({
                        "type": "tool_result",
                        "tool_use_id": block.id,
                        "content": tool_result[:5000]  # Limit result size
                    })
            
            messages.append({"role": "user", "content": tool_results})
        
        elif response.stop_reason == "end_turn":
            # Claude finished without tool call - might need prompting
            print("Claude stopped without completing. Prompting to continue...")
            messages.append({"role": "user", "content": "Please continue with the task. Use the task_complete tool when done."})
    
    # Max iterations reached
    result.error = f"Max iterations ({max_iterations}) reached"
    result.finished_at = datetime.now().isoformat()
    result.messages = [{"role": m["role"], "content": str(m["content"])[:500]} for m in messages]
    print(f"\n❌ Max iterations reached")
    return result

## System Prompts

In [6]:
SYSTEM_PROMPT_WITHOUT_LENSPR = """
You are a code assistant working on a Python project.

Available tools:
- read_file: Read file contents
- write_file: Write to files
- search_files: Search with grep
- list_files: List Python files
- run_tests: Run pytest
- task_complete: Call when done

Project root: lenspr/ (a Python code analysis library)

Rules:
1. Complete the task fully
2. Update ALL affected files
3. Run tests to verify
4. Call task_complete when done
"""

SYSTEM_PROMPT_WITH_LENSPR = """
You are a code assistant working on a Python project with LensPR code graph tools.

IMPORTANT: Use LensPR tools for Python code:
- lens_context: Get function + callers + callees + tests (use this first!)
- lens_check_impact: ALWAYS check before modifying code
- lens_find_usages: Find all references to a function
- lens_validate_change: Validate code before applying
- lens_update_node: Update a function's code
- lens_search: Search by name
- lens_grep: Search in code with graph context

Standard tools (for non-Python or when needed):
- read_file, write_file, search_files, list_files, run_tests
- task_complete: Call when done

Project root: lenspr/ (a Python code analysis library)

Rules:
1. Use lens_check_impact BEFORE any code changes
2. Use lens_context to understand functions
3. Complete the task fully
4. Run tests to verify
5. Call task_complete when done
"""

print("System prompts defined")

System prompts defined


## Evaluation Tasks

In [7]:
TASKS = {
    "task1_understand": {
        "name": "Understand a function",
        "prompt": """
Analyze the function `check_impact` in lenspr/graph.py.

Tell me:
1. What does it do?
2. What functions call it?
3. What functions does it call?
4. What tests cover it?

Call task_complete with your analysis.
""",
        "expected": ["get_impact_zone", "callers", "callees", "tests"]
    },
    
    "task2_find_usages": {
        "name": "Find all usages",
        "prompt": """
Find ALL places where the function `get_node` from lenspr/database.py is used.

List:
1. All files that import it
2. All functions that call it
3. All tests that use it

Call task_complete with the complete list.
""",
        "expected": ["callers", "importers"]
    },
    
    "task3_safe_change": {
        "name": "Safe code change",
        "prompt": """
I want to add a new parameter `include_external: bool = False` to the function 
`get_connections` in lenspr/database.py.

Before making ANY changes:
1. Analyze what will be affected by this change
2. List all callers that might need updates
3. Tell me the severity/risk of this change

DO NOT actually make the change - just analyze the impact.
Call task_complete with your analysis.
""",
        "expected": ["impact", "severity", "callers"]
    }
}

print(f"Defined {len(TASKS)} evaluation tasks")
for task_id, task in TASKS.items():
    print(f"  - {task_id}: {task['name']}")

Defined 3 evaluation tasks
  - task1_understand: Understand a function
  - task2_find_usages: Find all usages
  - task3_safe_change: Safe code change


---
## Run Benchmarks

In [8]:
def run_benchmark(task_id: str, with_lenspr: bool) -> BenchmarkResult:
    """Run a single benchmark task."""
    task = TASKS[task_id]
    mode = "with_lenspr" if with_lenspr else "without_lenspr"
    
    print(f"\n{'='*60}")
    print(f"Running: {task['name']} ({mode})")
    print(f"{'='*60}")
    
    tools = LENSPR_TOOLS if with_lenspr else STANDARD_TOOLS
    system = SYSTEM_PROMPT_WITH_LENSPR if with_lenspr else SYSTEM_PROMPT_WITHOUT_LENSPR
    
    result = run_agent(
        task=task["prompt"],
        tools=tools,
        system_prompt=system,
        max_iterations=15
    )
    
    result.task_id = task_id
    result.mode = mode
    
    # Save result
    result_path = RESULTS_DIR / f"{task_id}_{mode}.json"
    result.save(result_path)
    print(f"\nSaved: {result_path}")
    
    return result

### Task 1: Understand a Function

In [9]:
# Run WITHOUT LensPR
result_without = run_benchmark("task1_understand", with_lenspr=False)


Running: Understand a function (without_lenspr)

--- Iteration 1 ---
Tokens: +956 in, +96 out
Claude: I'll analyze the `check_impact` function in lenspr/graph.py for you. Let me start by examining the function and then trace its usage and dependencies.
Tool: read_file({"path": "lenspr/graph.py"}...)

--- Iteration 2 ---
Tokens: +2538 in, +97 out
Claude: The file seems to be cut off. Let me search for the `check_impact` function specifically:
Tool: search_files({"pattern": "def check_impact", "path": "lenspr"}...)

--- Iteration 3 ---
Tokens: +2694 in, +92 out
Claude: The `check_impact` function is actually in `lenspr/__init__.py`, not in `graph.py`. Let me examine it there:
Tool: read_file({"path": "lenspr/__init__.py"}...)

--- Iteration 4 ---
Tokens: +4309 in, +76 out
Claude: The file is cut off again. Let me search for the specific function and its surrounding context:
Tool: search_files({"pattern": "def check_impact"}...)

--- Iteration 5 ---
Tokens: +4469 in, +60 out
Tool: search

In [10]:
# Run WITH LensPR
result_with = run_benchmark("task1_understand", with_lenspr=True)


Running: Understand a function (with_lenspr)

--- Iteration 1 ---
Tokens: +4432 in, +183 out
Claude: I'll analyze the `check_impact` function for you. Let me start by getting comprehensive context about this function.
Tool: lens_context({"node_id": "lenspr.graph.check_impact", "depth": 2, "include_callers": true, "include_callees": tru...)

--- Iteration 2 ---
Tokens: +4699 in, +89 out
Claude: Let me search for the function first to find its exact node ID:
Tool: lens_search({"query": "check_impact", "search_in": "name"}...)

--- Iteration 3 ---
Tokens: +5210 in, +107 out
Claude: I see there are multiple functions related to `check_impact`. Let me look in the graph.py file specifically:
Tool: lens_list_nodes({"file_path": "lenspr/graph.py", "name": "check_impact"}...)

--- Iteration 4 ---
Tokens: +5387 in, +77 out
Claude: Let me check what functions are in the graph.py file:
Tool: lens_list_nodes({"file_path": "lenspr/graph.py"}...)

--- Iteration 5 ---
Tokens: +6473 in, +220 out
Claud

### Task 2: Find All Usages

In [11]:
# Run WITHOUT LensPR
result_without = run_benchmark("task2_find_usages", with_lenspr=False)


Running: Find all usages (without_lenspr)

--- Iteration 1 ---
Tokens: +951 in, +99 out
Claude: I'll help you find all places where the `get_node` function from `lenspr/database.py` is used. Let me start by exploring the project structure and then systematically search for all usages.
Tool: list_files({"path": "."}...)

--- Iteration 2 ---
Tokens: +3104 in, +66 out
Claude: Let me focus on the lenspr directory:
Tool: list_files({"path": "lenspr"}...)

--- Iteration 3 ---
Tokens: +3410 in, +79 out
Claude: Now let me check the database.py file to see the `get_node` function:
Tool: read_file({"path": "lenspr/database.py"}...)

--- Iteration 4 ---
Tokens: +4907 in, +96 out
Claude: Let me continue reading the database.py file to see the complete `get_node` function:
Tool: search_files({"pattern": "def get_node", "path": "lenspr"}...)

--- Iteration 5 ---
Tokens: +5136 in, +75 out
Claude: Now let me read the specific get_node function from database.py:
Tool: read_file({"path": "lenspr/databa

In [12]:
# Run WITH LensPR
result_with = run_benchmark("task2_find_usages", with_lenspr=True)


Running: Find all usages (with_lenspr)

--- Iteration 1 ---
Tokens: +4427 in, +104 out
Claude: I'll find all places where the `get_node` function from `lenspr/database.py` is used. Let me start by getting the context and then finding all usages.
Tool: lens_context({"node_id": "lenspr.database.get_node"}...)

--- Iteration 2 ---
Tokens: +6257 in, +97 out
Claude: Now let me find all usages of this function:
Tool: lens_find_usages({"node_id": "lenspr.database.get_node", "include_tests": true}...)

--- Iteration 3 ---
Tokens: +8065 in, +105 out
Claude: Let me also check for imports to get the complete picture:
Tool: lens_grep({"pattern": "from lenspr.database import.*get_node|import.*get_node.*from.*database", "max_results":...)

--- Iteration 4 ---
Tokens: +8451 in, +88 out
Claude: Now let me search more specifically for imports of get_node:
Tool: lens_grep({"pattern": "get_node", "max_results": 100}...)

--- Iteration 5 ---
Tokens: +10353 in, +88 out
Claude: Let me continue to get the r

### Task 3: Safe Code Change Analysis

In [13]:
# Run WITHOUT LensPR
result_without = run_benchmark("task3_safe_change", with_lenspr=False)


Running: Safe code change (without_lenspr)

--- Iteration 1 ---
Tokens: +990 in, +108 out
Claude: I'll analyze the impact of adding the `include_external: bool = False` parameter to the `get_connections` function in `lenspr/database.py`. Let me start by examining the current codebase structure and...
Tool: list_files({"path": "."}...)

--- Iteration 2 ---
Tokens: +3152 in, +74 out
Claude: Let me look at the actual project structure by listing files in the lenspr directory:
Tool: list_files({"path": "lenspr"}...)

--- Iteration 3 ---
Tokens: +3466 in, +82 out
Claude: Now let me examine the current `get_connections` function in `lenspr/database.py`:
Tool: read_file({"path": "lenspr/database.py"}...)

--- Iteration 4 ---
Tokens: +4966 in, +95 out
Claude: Let me see the rest of the file and specifically find the `get_connections` function:
Tool: search_files({"pattern": "def get_connections", "path": "lenspr"}...)

--- Iteration 5 ---
Tokens: +5122 in, +90 out
Claude: The function is not 

In [14]:
# Run WITH LensPR
result_with = run_benchmark("task3_safe_change", with_lenspr=True)


Running: Safe code change (with_lenspr)

--- Iteration 1 ---
Tokens: +4466 in, +128 out
Claude: I'll analyze the impact of adding the `include_external: bool = False` parameter to the `get_connections` function in `lenspr/database.py`. Let me start by examining the function and its usage.
Tool: lens_context({"node_id": "lenspr.database.get_connections", "depth": 2}...)

--- Iteration 2 ---
Tokens: +4678 in, +89 out
Claude: Let me search for the function first to find the correct node ID:
Tool: lens_search({"query": "get_connections", "search_in": "name"}...)

--- Iteration 3 ---
Tokens: +5085 in, +98 out
Claude: Let me look at the actual implementation in `lenspr/database.py`:
Tool: lens_list_nodes({"file_path": "lenspr/database.py", "type": "function"}...)

--- Iteration 4 ---
Tokens: +6578 in, +131 out
Claude: I don't see a `get_connections` function in `lenspr/database.py`. Let me check what function might be related - I see `get_edges` which seems like it might be the function you

---
## Results Analysis

In [15]:
def load_all_results() -> dict:
    """Load all benchmark results."""
    results = {}
    for path in RESULTS_DIR.glob("*.json"):
        result = BenchmarkResult.load(path)
        key = f"{result.task_id}_{result.mode}"
        results[key] = result
    return results

def compare_results(task_id: str, results: dict):
    """Compare with vs without LensPR for a task."""
    with_key = f"{task_id}_with_lenspr"
    without_key = f"{task_id}_without_lenspr"
    
    if with_key not in results or without_key not in results:
        print(f"Missing results for {task_id}")
        return
    
    w = results[with_key]
    wo = results[without_key]
    
    task_name = TASKS[task_id]["name"]
    
    print(f"\n{'='*60}")
    print(f"Task: {task_name}")
    print(f"{'='*60}")
    print(f"")
    print(f"{'Metric':<25} {'Without LensPR':>15} {'With LensPR':>15} {'Diff':>10}")
    print(f"{'-'*65}")
    
    # Input tokens
    diff = wo.total_input_tokens - w.total_input_tokens
    pct = (diff / wo.total_input_tokens * 100) if wo.total_input_tokens else 0
    print(f"{'Input tokens':<25} {wo.total_input_tokens:>15,} {w.total_input_tokens:>15,} {pct:>+9.1f}%")
    
    # Output tokens
    diff = wo.total_output_tokens - w.total_output_tokens
    pct = (diff / wo.total_output_tokens * 100) if wo.total_output_tokens else 0
    print(f"{'Output tokens':<25} {wo.total_output_tokens:>15,} {w.total_output_tokens:>15,} {pct:>+9.1f}%")
    
    # Iterations
    print(f"{'Iterations':<25} {wo.iterations:>15} {w.iterations:>15} {w.iterations - wo.iterations:>+10}")
    
    # Tool calls
    print(f"{'Tool calls':<25} {wo.tool_call_count:>15} {w.tool_call_count:>15} {w.tool_call_count - wo.tool_call_count:>+10}")
    
    # Completed
    print(f"{'Completed':<25} {str(wo.completed):>15} {str(w.completed):>15}")
    
    # Tool breakdown
    print(f"\nTool usage:")
    print(f"  Without: {', '.join(t['name'] for t in wo.tool_calls[:10])}")
    print(f"  With:    {', '.join(t['name'] for t in w.tool_calls[:10])}")

In [16]:
# Load and compare all results
results = load_all_results()
print(f"Loaded {len(results)} results")

for task_id in TASKS.keys():
    compare_results(task_id, results)

Loaded 6 results

Task: Understand a function

Metric                     Without LensPR     With LensPR       Diff
-----------------------------------------------------------------
Input tokens                       89,800         117,649     -31.0%
Output tokens                       1,200           2,199     -83.2%
Iterations                             15              13         -2
Tool calls                             15              13         -2
Completed                           False            True

Tool usage:
  Without: read_file, search_files, read_file, search_files, search_files, search_files, search_files, search_files, search_files, list_files
  With:    lens_context, lens_search, lens_list_nodes, lens_list_nodes, lens_context, lens_find_usages, lens_context, lens_context, lens_search, lens_get_node

Task: Find all usages

Metric                     Without LensPR     With LensPR       Diff
-----------------------------------------------------------------
Input token

## Summary Table

In [17]:
def summary_table(results: dict):
    """Generate summary comparison table."""
    print("\n" + "="*80)
    print("SUMMARY: LensPR Impact on Claude Code Performance")
    print("="*80)
    print()
    
    totals = {"with": {"in": 0, "out": 0, "iter": 0, "tools": 0},
              "without": {"in": 0, "out": 0, "iter": 0, "tools": 0}}
    
    print(f"{'Task':<30} {'Mode':<15} {'Input':>10} {'Output':>10} {'Iter':>6} {'Tools':>6}")
    print("-" * 80)
    
    for task_id in TASKS.keys():
        for mode in ["without_lenspr", "with_lenspr"]:
            key = f"{task_id}_{mode}"
            if key in results:
                r = results[key]
                mode_short = "WITHOUT" if "without" in mode else "WITH"
                print(f"{TASKS[task_id]['name']:<30} {mode_short:<15} {r.total_input_tokens:>10,} {r.total_output_tokens:>10,} {r.iterations:>6} {r.tool_call_count:>6}")
                
                m = "without" if "without" in mode else "with"
                totals[m]["in"] += r.total_input_tokens
                totals[m]["out"] += r.total_output_tokens
                totals[m]["iter"] += r.iterations
                totals[m]["tools"] += r.tool_call_count
        print()
    
    print("-" * 80)
    print(f"{'TOTAL':<30} {'WITHOUT':<15} {totals['without']['in']:>10,} {totals['without']['out']:>10,} {totals['without']['iter']:>6} {totals['without']['tools']:>6}")
    print(f"{'TOTAL':<30} {'WITH':<15} {totals['with']['in']:>10,} {totals['with']['out']:>10,} {totals['with']['iter']:>6} {totals['with']['tools']:>6}")
    
    # Calculate savings
    if totals['without']['in'] > 0:
        in_savings = (totals['without']['in'] - totals['with']['in']) / totals['without']['in'] * 100
        print(f"\n📊 Input token savings: {in_savings:.1f}%")
    if totals['without']['iter'] > 0:
        iter_savings = (totals['without']['iter'] - totals['with']['iter']) / totals['without']['iter'] * 100
        print(f"📊 Iteration savings: {iter_savings:.1f}%")

summary_table(results)


SUMMARY: LensPR Impact on Claude Code Performance

Task                           Mode                 Input     Output   Iter  Tools
--------------------------------------------------------------------------------
Understand a function          WITHOUT             89,800      1,200     15     15
Understand a function          WITH               117,649      2,199     13     13

Find all usages                WITHOUT            113,004      1,286     15     15
Find all usages                WITH               108,364      1,760     10     10

Safe code change               WITHOUT            112,030      1,410     15     15
Safe code change               WITH                96,492      1,760     11     11

--------------------------------------------------------------------------------
TOTAL                          WITHOUT            314,834      3,896     45     45
TOTAL                          WITH               322,505      5,719     34     34

📊 Input token savings: -2.4%
📊 Iter